# U(2) and its corresponding rotation

As is widely known, $SU(2)/ Z_2 \cong SO(3)$

More generally, any 3-D rotation can be regarded as a group of Unitary Matrices, differing by a global phase.

In this notebook, I am trying to give some useful functions to do the transformation between different representations.

## · $\{\hat{n},\theta\}$  ——   The rotation axis and angle
## · The matrix $U$, represented by $R_{\hat{n}} (\theta)$.      
####   $\ \ \ $     Here        $ U = R_{\hat{n}} (\theta) e^{i\alpha} $
## · The ZXZ Euler Angle $\{\beta,\gamma,\delta\}$ , and the phase $\alpha $

.

###  1.  $\{\hat{n},\theta\} \rightarrow R_{\hat{n}} (\theta)$
##### From rotation axis and angle, constructing SU(2) rotation matrix 

In [54]:
import numpy as np
from scipy.linalg import expm
import math

In [55]:
sigma_x = np.array([[0, 1], [1, 0]], dtype=complex)
sigma_y = np.array([[0, -1j], [1j, 0]], dtype=complex)
sigma_z = np.array([[1, 0], [0, -1]], dtype=complex)

In [56]:
def rotation_to_matrix_U2(n, theta):
    """
    Calculate the rotation matrix for a rotation around axis n by angle theta
    --------------
    Parameters:
    n: Rotation axis (n_x, n_y, n_z), should be a normalized vector
    theta: Rotation angle (in radians)
    --------------
    return: 2x2 rotation matrix
    """
    # Normalize the rotation axis
    n = n / np.linalg.norm(n)
    
    # Calculate n · σ (linear combination of Pauli matrices)
    n_dot_sigma = n[0] * sigma_x + n[1] * sigma_y + n[2] * sigma_z
    
    # Calculate the rotation matrix: R_n(θ) = exp(-i * θ / 2 * n · σ)
    rotation = expm(-1j * theta / 2 * n_dot_sigma)
    
    return rotation

###  2.  Matrix Elements $ \{a,b,c,d\} \rightarrow$ ZXZ Euler Angle $\{\beta,\gamma,\delta\}$ , and the phase $\alpha $

In [57]:
def calculate_angle_from_complex(z):
    """
    Calculate the angle (phase) from a complex number.
    
    Parameters:
    z (complex): A complex number
    
    Returns:
    float: The angle (phase) in the range [0, 2*pi]
    """
    # Extract the real and imaginary parts of the complex number
    a = z.real
    b = z.imag
    
    # Use atan2 to calculate the phase angle, result will be in the range [-pi, pi]
    theta = math.atan2(b, a)
    
    # Convert the angle to the range [0, 2*pi]
    if theta < 0:
        theta += 2 * math.pi
        
    return theta

In [58]:
def matrix_U2_to_EulerAngle(U):
    """
    Calculate Euler angles from a 2x2 SU(2) matrix
    --------------
    Parameters:
    U: 2x2 SU(2) matrix
    --------------
    return: Euler angles (β, γ,δ),and phase factor α
    """
    a,b,c,d = U[0,0],U[0,1],U[1,0],U[1,1]
    γ = 2*np.arccos(abs(a))
    if np.isclose(γ, np.pi):
        β = 0
        δ = calculate_angle_from_complex(b/c)
        α = calculate_angle_from_complex(1j*b*np.exp(-1j*0.5*δ))
    elif np.isclose(γ, 0):
        β = 0
        δ = calculate_angle_from_complex(d/a)
        α = calculate_angle_from_complex(a*np.exp(1j*0.5*δ))
    else:
        β = calculate_angle_from_complex(1j*c/(a*np.tan(0.5*γ)))
        δ = calculate_angle_from_complex(1j*b/(a*np.tan(0.5*γ)))
        α = calculate_angle_from_complex(a*(1/np.cos(0.5*γ))*np.exp(1j*0.5*(β+δ)))
    return β, γ, δ, α


### 3.  ZXZ Euler Angle $\{\beta,\gamma,\delta\}$ , and the phase $\alpha$ $\rightarrow$ Matrix Elements $ \{a,b,c,d\}$

This part is easy, you just do the multiplication.

In [59]:
def EulerAngle_to_matrix_U2(β, γ, δ, α):
    """
    Calculate a 2x2 SU(2) matrix from Euler angles
    --------------
    Parameters:
    β: Euler angle β
    γ: Euler angle γ
    δ: Euler angle δ
    α: Phase factor α
    --------------
    return: 2x2 SU(2) matrix
    """
    Rz1 = rotation_to_matrix_U2([0, 0, 1], β)
    Rx = rotation_to_matrix_U2([1, 0, 0], γ)
    Rz2 = rotation_to_matrix_U2([0, 0, 1], δ)
    phase = np.exp(1j*α)
    U = Rz1 @ Rx @ Rz2*phase
    return U

###  4.  $ U \rightarrow \{\hat{n},\theta\}$ ,and the phase $\alpha$
##### The inverse process of '1'
##### Given the matrix U, give out the axis and angle, as well as a phase $\alpha$

Here, we choose theta in [0,pi] to determine the only axis

In [73]:
def matrix_U2_to_rotation(U):
    """
    Calculate the rotation axis and angle from a 2x2 SU(2) matrix
    --------------
    Parameters:
    U: 2x2 SU(2) matrix
    --------------
    return: Rotation axis (n_x, n_y, n_z) and rotation angle θ, as well as the phase factor α
    """
    
    a,b,c,d = U[0,0],U[0,1],U[1,0],U[1,1]
    theta = 2*np.arccos(abs((a+d)/2))
    if np.isclose(theta, np.pi):   # See if theta is pi
        n_z = abs(a)
        if np.isclose(n_z, 0):   # unable to determine alpha
            n_y = abs(c-b)/2
            if np.isclose(n_y, 0):
                n_x = 1
                alpha = calculate_angle_from_complex((c+b)/(-2j))
            else:
                alpha = calculate_angle_from_complex((c-b)/(2*n_y))
                n_x = (c+b)/(-2j*np.exp(1j*alpha))

        else:
            alpha = calculate_angle_from_complex(a/(-1j*n_z))
            n_y = (c-b)/(2*np.exp(1j*alpha))
            n_x = (c+b)/(-2j*np.exp(1j*alpha))
    elif np.isclose(theta, 0):
        # Not a rotation at all, random axis
        n_z = 1
        n_y = 0
        n_x = 0
        thata = 0
        alpha = calculate_angle_from_complex(a)
    else:
        alpha = calculate_angle_from_complex((a+d)/(2*np.cos(theta/2)))
        phase_destroy = np.exp(-1j*alpha)
        n_x = (b+c)/(-2j*np.sin(theta/2))*phase_destroy
        n_y = (c-b)/(2*np.sin(theta/2))*phase_destroy
        n_z = (d-a)/(2j*np.sin(theta/2)) *phase_destroy
    
    n_x = np.real(n_x)
    n_y = np.real(n_y)
    n_z = np.real(n_z)
    mod = np.sqrt(n_x**2 + n_y**2 + n_z**2)
    n_x = n_x/mod
    n_y = n_y/mod
    n_z = n_z/mod

    return n_x, n_y, n_z, theta, alpha


   

### 5. Given an axis and an angle, we can use the combination of the predefined function to get the Euler Angle

In [82]:
def rotation_to_EulerAngle(n, theta):
    """
    Calculate Euler angles from a rotation around axis n by angle theta
    --------------
    Parameters:
    n: Rotation axis (n_x, n_y, n_z), should be a normalized vector
    theta: Rotation angle (in radians)
    --------------
    return: Euler angles (β, γ,δ),and phase factor α
    """
    R_n_theta = rotation_to_matrix_U2(n, theta)
    return matrix_U2_to_EulerAngle(R_n_theta)

### 6.Given the Euler Angle, we can get the axis and the angle of rotation.

In [ ]:
def EulerAngle_to_rotation(β, γ, δ, α):
    """
    Calculate the rotation axis and angle from Euler angles
    --------------
    Parameters:
    β: Euler angle β
    γ: Euler angle γ
    δ: Euler angle δ
    α: Phase factor α
    --------------
    return: Rotation axis (n_x, n_y, n_z) and rotation angle θ, as well as the phase factor α
    """
    U = EulerAngle_to_matrix_U2(β, γ, δ, α)
    return matrix_U2_to_rotation(U)

## To do: Maybe we can build the relationship between 2*2 Unitary matrices with 3D matrices too

It's easy. Consider the euler angle and everything is so easy!